In [ ]:
import pandas as pd
import numpy as np
# transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# nlp
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!apt install -qq enchant
!pip install pyenchant
import enchant # pip install pyenchant
import spacy

The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,312 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspe

# Utility functions

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [ ]:
def split_segment_words(chars, exclude=None, dictionary = enchant.Dict("en_US")):
    """
    Segment a string of chars using the pyenchant vocabulary.
    Keeps longest possible words that account for all characters,
    and returns list of segmented words.

    :param chars: (str) The character string to segment.
    :param exclude: (set) A set of string to exclude from consideration.
                    (These have been found previously to lead to dead ends.)
                    If an excluded word occurs later in the string, this
                    function will fail.
    """
    words = []

    if not chars.isalpha():  # don't check punctuation etc.; needs more work
        return [chars]

    if not exclude:
        exclude = set()

    working_chars = chars
    while working_chars:
        # iterate through segments of the chars starting with the longest segment possible
        for i in range(len(working_chars), 1, -1):
            segment = working_chars[:i]
            if dictionary.check(segment) and segment not in exclude:
                words.append(segment)
                working_chars = working_chars[i:]
                break
        else:  # no matching segments were found
            if words:
                exclude.add(words[-1])
                return split_segment_words(chars, exclude=exclude)
            # let the user know a word was missing from the dictionary,
            # but keep the word
            # print('"{chars}" not in dictionary (so just keeping as one segment)!'
            #       .format(chars=chars))
            return [chars]
    # return a list of words based on the segmentation
    return words

In [ ]:
def split_words(str_arr):
  list_split_words = []
  for chars in str_arr:
    split_word = split_segment_words(chars)
    list_split_words.extend(split_word)
  return list_split_words

# processing fake job posts

In [ ]:
url_fake = 'https://drive.google.com/file/d/1zsgb5IvXtkOUsFsv7-xaiB9_GXrYiVA3/view?usp=sharing'
url_fake='https://drive.google.com/uc?id=' + url_fake.split('/')[-2]

data_fake_posts = pd.read_csv(url_fake)

### data value counts

In [ ]:
data_fake_posts['title'].value_counts()

English Teacher Abroad                                         311
Customer Service Associate                                     146
Graduates: English Teacher Abroad (Conversational)             144
English Teacher Abroad                                          95
Software Engineer                                               86
                                                              ... 
West Coast Regional Channel Manager (RCM)                        1
BI Practice Manager                                              1
Community Coordinator- Ambassador Programme                      1
Senior Traffic Engineer                                          1
Project Cost Control Staff Engineer - Cost Control Exp - TX      1
Name: title, Length: 11231, dtype: int64

In [ ]:
data_fake_posts['department'].value_counts()

Sales                                          551
Engineering                                    487
Marketing                                      401
Operations                                     270
IT                                             225
                                              ... 
Commercial Management / Contract Management      1
Exec                                             1
Marcomm                                          1
CRM                                              1
Hospitality                                      1
Name: department, Length: 1337, dtype: int64

In [ ]:
data_fake_posts['requirements'].value_counts()

University degree required. TEFL / TESOL / CELTA or teaching experience preferred but not necessaryCanada/US passport holders only                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
data_fake_posts['description'].value_counts()

Play with kids, get paid for it Love travel? Jobs in Asia$1,500+ USD monthly ($200 Cost of living)Housing provided (Private/Furnished)Airfare ReimbursedExcellent for student loans/credit cardsGabriel Adkins : #URL_ed9094c60184b8a4975333957f05be37e69d3cdb68decc9dd9a4242733cfd7f7##URL_75db76d58f7994c7db24e8998c2fc953ab9a20ea9ac948b217693963f78d2e6b#12 month contract : Apply today                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     379
Play with kids, get paid for it :-)Love travel? Jobs in Asia$1500 USD + monthly ($200 Cost of living)Housing providedAirfare ReimbursedExcellent fo

In [ ]:
data_fake_posts['industry'].value_counts()

Information Technology and Services    1734
Computer Software                      1376
Internet                               1062
Marketing and Advertising               828
Education Management                    822
                                       ... 
Shipbuilding                              1
Sporting Goods                            1
Museums and Institutions                  1
Wine and Spirits                          1
Ranching                                  1
Name: industry, Length: 131, dtype: int64

In [ ]:
data_fake_posts['employment_type'].value_counts()

Full-time    11620
Contract      1524
Part-time      797
Temporary      241
Other          227
Name: employment_type, dtype: int64

In [ ]:
data_fake_posts['required_education'].value_counts()

Bachelor's Degree                    5145
High School or equivalent            2080
Unspecified                          1397
Master's Degree                       416
Associate Degree                      274
Certification                         170
Some College Coursework Completed     102
Professional                           74
Vocational                             49
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree                     6
Name: required_education, dtype: int64

In [ ]:
data_fake_posts['required_experience'].value_counts()

Mid-Senior level    3809
Entry level         2697
Associate           2297
Not Applicable      1116
Director             389
Internship           381
Executive            141
Name: required_experience, dtype: int64

In [ ]:
data_fake_posts.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

### stop words → lemmatization → stem

In [ ]:
# filter null rows
data_fake_posts_trimmed = data_fake_posts[['title','department','description','requirements',
        'required_experience', 'required_education','employment_type',
        'fraudulent']].loc[data_fake_posts['fraudulent']==0].drop(columns=['fraudulent'])

filters = ['required_education', 'required_experience', 'employment_type', 'requirements']

data_fake_posts_filtered = data_fake_posts_trimmed
for filter in filters:
  filtered = data_fake_posts_trimmed[filter].isnull()
  data_fake_posts_filtered = data_fake_posts_filtered.loc[~filtered]
  
data_fake_posts_filtered = data_fake_posts_filtered.apply(lambda x: x.astype(str).str.lower())

In [ ]:
data_fake_posts_filtered

,title,department,description,requirements,required_experience,required_education,employment_type
3,account executive - washington dc,sales,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",mid-senior level,bachelor's degree,full-time
4,bill review manager,nan,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,mid-senior level,bachelor's degree,full-time
6,head of content (m/f),androidpit,your responsibilities: manage the english-spea...,your know-how: ...,mid-senior level,master's degree,full-time
9,customer service associate - part time,nan,the customer service associate will be based i...,minimum requirements:minimum of 6 months custo...,entry level,high school or equivalent,part-time
10,asp.net developer job opportunity at united st...,nan,position : #url_86fd830a95a64e2b30ceed829e63fd...,position : #url_86fd830a95a64e2b30ceed829e63fd...,mid-senior level,bachelor's degree,full-time
...,...,...,...,...,...,...,...
17865,portfolio development associate - paris & rest...,sales,"avenuestory is looking for a curious, ambitiou...","comfortable on the phone, fluent in french and...",entry level,bachelor's degree,full-time
17867,admin assistant,sales,we are seeking an administrator to work in a b...,proficient experience in microsoft word and ex...,entry level,unspecified,full-time
17870,senior financial analyst (retail),nan,we are activity searching for a senior financi...,must have a minimum of 3 + years in financial ...,mid-senior level,bachelor's degree,full-time
17876,payroll accountant,accounting,the payroll accountant will focus primarily on...,- b.a. or b.s. in accounting- desire to have f...,mid-senior level,bachelor's degree,full-time


In [ ]:
data_fake_posts_filtered.reset_index(inplace = True)
data_fake_posts_filtered.drop('index', axis=1, inplace=True)

In [ ]:
# Load the en_core_web_sm model
nlp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS
# lemmatize job description
tokenized_description = []
for description in data_fake_posts_filtered['description']: 
    lemmas = [token.lemma_ for token in nlp(description)]
    tokenized_description.append([lemma.lower() for lemma in lemmas if lemma.isalpha() and lemma not in stopwords or lemma == '-PRON-'])
data_fake_posts_filtered['description_cleaned'] = tokenized_description

In [ ]:
# lemmatize job requirements
tokenized_requirements = []
for requirement in data_fake_posts_filtered['requirements']: 
  lemmas = [token.lemma_ for token in nlp(requirement)]
  tokenized_requirements.append([lemma.lower() for lemma in lemmas if lemma.isalpha() and lemma not in stopwords or lemma == '-PRON-'])
data_fake_posts_filtered['requirements_cleaned'] = tokenized_requirements


In [ ]:
data_fake_posts_filtered.reset_index(inplace = True)
data_fake_posts_filtered.drop('index', axis=1, inplace=True)
data_fake_posts_filtered

,title,department,description,requirements,required_experience,required_education,employment_type,description_cleaned,requirements_cleaned
0,account executive - washington dc,sales,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",mid-senior level,bachelor's degree,full-time,"[company, esri, environmental, systems, resear...","[education, bachelor, master, gis, business, a..."
1,bill review manager,nan,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,mid-senior level,bachelor's degree,full-time,"[job, title, itemization, review, managerlocat...","[qualification, rn, license, state, texasdiplo..."
2,head of content (m/f),androidpit,your responsibilities: manage the english-spea...,your know-how: ...,mid-senior level,master's degree,full-time,"[-pron-, responsibility, manage, english, spea...","[-pron-, know, university, college, degree, jo..."
3,customer service associate - part time,nan,the customer service associate will be based i...,minimum requirements:minimum of 6 months custo...,entry level,high school or equivalent,part-time,"[customer, service, associate, base, phoenix, ...","[minimum, requirement, minimum, month, custome..."
4,asp.net developer job opportunity at united st...,nan,position : #url_86fd830a95a64e2b30ceed829e63fd...,position : #url_86fd830a95a64e2b30ceed829e63fd...,mid-senior level,bachelor's degree,full-time,"[position, developerjob, location, united, sta...","[position, developerjob, location, united, sta..."
...,...,...,...,...,...,...,...,...,...
7530,portfolio development associate - paris & rest...,sales,"avenuestory is looking for a curious, ambitiou...","comfortable on the phone, fluent in french and...",entry level,bachelor's degree,full-time,"[avenuestory, look, curious, ambitious, adapta...","[comfortable, phone, fluent, french, english, ..."
7531,admin assistant,sales,we are seeking an administrator to work in a b...,proficient experience in microsoft word and ex...,entry level,unspecified,full-time,"[-pron-, seek, administrator, work, busy, sale...","[proficient, experience, microsoft, word, exce..."
7532,senior financial analyst (retail),nan,we are activity searching for a senior financi...,must have a minimum of 3 + years in financial ...,mid-senior level,bachelor's degree,full-time,"[-pron-, activity, search, senior, financial, ...","[minimum, year, financial, planning, analysis,..."
7533,payroll accountant,accounting,the payroll accountant will focus primarily on...,- b.a. or b.s. in accounting- desire to have f...,mid-senior level,bachelor's degree,full-time,"[payroll, accountant, focus, primarily, payrol...","[desire, fun, -pron-, genuine, passion, accoun..."


In [ ]:
# remove non-english job posts
print("before: " + str(len(data_fake_posts_filtered)))
for i, description in enumerate(data_fake_posts_filtered['description_cleaned']):
  for word in description:
    if isEnglish(word) == False:
      data_fake_posts_filtered = data_fake_posts_filtered.drop(i, axis=0)
      break
print("after: " + str(len(data_fake_posts_filtered)))

before: 7535
after: 7459


In [ ]:
data_fake_posts_filtered.reset_index(inplace = True)
data_fake_posts_filtered.drop('index', axis=1, inplace=True)
# data_fake_posts_filtered.drop('level_0', axis=1, inplace=True)
data_fake_posts_filtered

,title,department,description,requirements,required_experience,required_education,employment_type,description_cleaned,requirements_cleaned
0,account executive - washington dc,sales,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",mid-senior level,bachelor's degree,full-time,"[company, esri, environmental, systems, resear...","[education, bachelor, master, gis, business, a..."
1,bill review manager,nan,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,mid-senior level,bachelor's degree,full-time,"[job, title, itemization, review, managerlocat...","[qualification, rn, license, state, texasdiplo..."
2,head of content (m/f),androidpit,your responsibilities: manage the english-spea...,your know-how: ...,mid-senior level,master's degree,full-time,"[-pron-, responsibility, manage, english, spea...","[-pron-, know, university, college, degree, jo..."
3,customer service associate - part time,nan,the customer service associate will be based i...,minimum requirements:minimum of 6 months custo...,entry level,high school or equivalent,part-time,"[customer, service, associate, base, phoenix, ...","[minimum, requirement, minimum, month, custome..."
4,asp.net developer job opportunity at united st...,nan,position : #url_86fd830a95a64e2b30ceed829e63fd...,position : #url_86fd830a95a64e2b30ceed829e63fd...,mid-senior level,bachelor's degree,full-time,"[position, developerjob, location, united, sta...","[position, developerjob, location, united, sta..."
...,...,...,...,...,...,...,...,...,...
7454,portfolio development associate - paris & rest...,sales,"avenuestory is looking for a curious, ambitiou...","comfortable on the phone, fluent in french and...",entry level,bachelor's degree,full-time,"[avenuestory, look, curious, ambitious, adapta...","[comfortable, phone, fluent, french, english, ..."
7455,admin assistant,sales,we are seeking an administrator to work in a b...,proficient experience in microsoft word and ex...,entry level,unspecified,full-time,"[-pron-, seek, administrator, work, busy, sale...","[proficient, experience, microsoft, word, exce..."
7456,senior financial analyst (retail),nan,we are activity searching for a senior financi...,must have a minimum of 3 + years in financial ...,mid-senior level,bachelor's degree,full-time,"[-pron-, activity, search, senior, financial, ...","[minimum, year, financial, planning, analysis,..."
7457,payroll accountant,accounting,the payroll accountant will focus primarily on...,- b.a. or b.s. in accounting- desire to have f...,mid-senior level,bachelor's degree,full-time,"[payroll, accountant, focus, primarily, payrol...","[desire, fun, -pron-, genuine, passion, accoun..."


In [ ]:
print("before: " + str(len(data_fake_posts_filtered)))
for i, description in enumerate(data_fake_posts_filtered['requirements_cleaned']):
  for word in description:
    if isEnglish(word) == False:
      data_fake_posts_filtered = data_fake_posts_filtered.drop(i, axis=0)
      break
print("after: " + str(len(data_fake_posts_filtered)))

before: 7459
after: 7443


In [ ]:
data_fake_posts_filtered.reset_index(inplace = True)
data_fake_posts_filtered.drop('index', axis=1, inplace=True)
data_fake_posts_filtered

,title,department,description,requirements,required_experience,required_education,employment_type,description_cleaned,requirements_cleaned
0,account executive - washington dc,sales,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",mid-senior level,bachelor's degree,full-time,"[company, esri, environmental, systems, resear...","[education, bachelor, master, gis, business, a..."
1,bill review manager,nan,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,mid-senior level,bachelor's degree,full-time,"[job, title, itemization, review, managerlocat...","[qualification, rn, license, state, texasdiplo..."
2,head of content (m/f),androidpit,your responsibilities: manage the english-spea...,your know-how: ...,mid-senior level,master's degree,full-time,"[-pron-, responsibility, manage, english, spea...","[-pron-, know, university, college, degree, jo..."
3,customer service associate - part time,nan,the customer service associate will be based i...,minimum requirements:minimum of 6 months custo...,entry level,high school or equivalent,part-time,"[customer, service, associate, base, phoenix, ...","[minimum, requirement, minimum, month, custome..."
4,asp.net developer job opportunity at united st...,nan,position : #url_86fd830a95a64e2b30ceed829e63fd...,position : #url_86fd830a95a64e2b30ceed829e63fd...,mid-senior level,bachelor's degree,full-time,"[position, developerjob, location, united, sta...","[position, developerjob, location, united, sta..."
...,...,...,...,...,...,...,...,...,...
7438,portfolio development associate - paris & rest...,sales,"avenuestory is looking for a curious, ambitiou...","comfortable on the phone, fluent in french and...",entry level,bachelor's degree,full-time,"[avenuestory, look, curious, ambitious, adapta...","[comfortable, phone, fluent, french, english, ..."
7439,admin assistant,sales,we are seeking an administrator to work in a b...,proficient experience in microsoft word and ex...,entry level,unspecified,full-time,"[-pron-, seek, administrator, work, busy, sale...","[proficient, experience, microsoft, word, exce..."
7440,senior financial analyst (retail),nan,we are activity searching for a senior financi...,must have a minimum of 3 + years in financial ...,mid-senior level,bachelor's degree,full-time,"[-pron-, activity, search, senior, financial, ...","[minimum, year, financial, planning, analysis,..."
7441,payroll accountant,accounting,the payroll accountant will focus primarily on...,- b.a. or b.s. in accounting- desire to have f...,mid-senior level,bachelor's degree,full-time,"[payroll, accountant, focus, primarily, payrol...","[desire, fun, -pron-, genuine, passion, accoun..."


In [ ]:
# expand segment words. eg.verygood
for i, description in enumerate(data_fake_posts_filtered['description_cleaned']):
  words = split_words(description)
  data_fake_posts_filtered['description_cleaned'][i] = words

In [ ]:
for i, requirement in enumerate(data_fake_posts_filtered['requirements_cleaned']):
  words = split_words(requirement)
  data_fake_posts_filtered['requirements_cleaned'][i] = words

In [ ]:
# for i in range(0, len(data_fake_posts_filtered)):
#   data_fake_posts_filtered['description_cleaned'][i] = ' '.join(data_fake_posts_filtered['description_cleaned'][i])
#   data_fake_posts_filtered['requirements_cleaned'][i] = ' '.join(data_fake_posts_filtered['requirements_cleaned'][i])

In [ ]:
data_fake_posts_filtered

,title,department,description,requirements,required_experience,required_education,employment_type,description_cleaned,requirements_cleaned
0,account executive - washington dc,sales,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",mid-senior level,bachelor's degree,full-time,"[company, esri, environmental, systems, resear...","[education, bachelor, master, gis, business, a..."
1,bill review manager,nan,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,mid-senior level,bachelor's degree,full-time,"[job, title, itemization, review, manager, loc...","[qualification, rn, license, state, texasdiplo..."
2,head of content (m/f),androidpit,your responsibilities: manage the english-spea...,your know-how: ...,mid-senior level,master's degree,full-time,"[-pron-, responsibility, manage, english, spea...","[-pron-, know, university, college, degree, jo..."
3,customer service associate - part time,nan,the customer service associate will be based i...,minimum requirements:minimum of 6 months custo...,entry level,high school or equivalent,part-time,"[customer, service, associate, base, phoenix, ...","[minimum, requirement, minimum, month, custome..."
4,asp.net developer job opportunity at united st...,nan,position : #url_86fd830a95a64e2b30ceed829e63fd...,position : #url_86fd830a95a64e2b30ceed829e63fd...,mid-senior level,bachelor's degree,full-time,"[position, developer, job, location, united, s...","[position, developer, job, location, united, s..."
...,...,...,...,...,...,...,...,...,...
7438,portfolio development associate - paris & rest...,sales,"avenuestory is looking for a curious, ambitiou...","comfortable on the phone, fluent in french and...",entry level,bachelor's degree,full-time,"[avenue, story, look, curious, ambitious, adap...","[comfortable, phone, fluent, french, english, ..."
7439,admin assistant,sales,we are seeking an administrator to work in a b...,proficient experience in microsoft word and ex...,entry level,unspecified,full-time,"[-pron-, seek, administrator, work, busy, sale...","[proficient, experience, micros, oft, word, ex..."
7440,senior financial analyst (retail),nan,we are activity searching for a senior financi...,must have a minimum of 3 + years in financial ...,mid-senior level,bachelor's degree,full-time,"[-pron-, activity, search, senior, financial, ...","[minimum, year, financial, planning, analysis,..."
7441,payroll accountant,accounting,the payroll accountant will focus primarily on...,- b.a. or b.s. in accounting- desire to have f...,mid-senior level,bachelor's degree,full-time,"[payroll, accountant, focus, primarily, payrol...","[desire, fun, -pron-, genuine, passion, accoun..."


### export cleaned fake_job_posting

In [ ]:
from google.colab import files
data_fake_posts_filtered.to_csv('fake_job_posting_cleaned.csv', encoding = 'utf-8-sig') 
files.download('fake_job_posting_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# load cleaned fake_job_posting(files in snapshot folder)

In [ ]:
url_cleaned = 'https://drive.google.com/file/d/1TQiLUY1f9bLcjBiJxiNs9yZ2TllMrT4A/view?usp=sharing'
url_cleaned ='https://drive.google.com/uc?id=' + url_cleaned.split('/')[-2]

data_cleaned = pd.read_csv(url_cleaned)
data_cleaned.drop('Unnamed: 0', axis=1, inplace=True)
data_cleaned

,title,department,description,requirements,required_experience,required_education,employment_type,description_cleaned,requirements_cleaned
0,Account Executive - Washington DC,Sales,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Mid-Senior level,Bachelor's Degree,Full-time,company esri environmental systems research in...,education bachelor master gis business adminis...
1,Bill Review Manager,NaN,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Mid-Senior level,Bachelor's Degree,Full-time,job title itemization review manager location ...,qualification rn license state texasdiploma ba...
2,Head of Content (m/f),ANDROIDPIT,Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Mid-Senior level,Master's Degree,Full-time,-pron- responsibility manage english speak edi...,-pron- know how university college degree jour...
3,Customer Service Associate - Part Time,NaN,The Customer Service Associate will be based i...,Minimum Requirements:Minimum of 6 months custo...,Entry level,High School or equivalent,Part-time,customer service associate base phoenix az rig...,minimum requirement minimum month customer ser...
4,ASP.net Developer Job opportunity at United St...,NaN,Position : #URL_86fd830a95a64e2b30ceed829e63fd...,Position : #URL_86fd830a95a64e2b30ceed829e63fd...,Mid-Senior level,Bachelor's Degree,Full-time,position developer job location united states ...,position developer job location united states ...
...,...,...,...,...,...,...,...,...,...
7437,Portfolio Development Associate - Paris & rest...,Sales,"AvenueStory is looking for a curious, ambitiou...","Comfortable on the phone, fluent in French and...",Entry level,Bachelor's Degree,Full-time,avenue story look curious ambitious adaptable ...,comfortable phone fluent french english oral w...
7438,Admin Assistant,Sales,We are seeking an administrator to work in a b...,Proficient experience in Microsoft Word and Ex...,Entry level,Unspecified,Full-time,-pron- seek administrator work busy sale envir...,proficient experience micros oft word excel ac...
7439,Senior Financial Analyst (Retail),NaN,We are activity searching for a Senior Financi...,Must have a minimum of 3 + years in Financial ...,Mid-Senior level,Bachelor's Degree,Full-time,-pron- activity search senior financial analys...,must minimum year financial planning analysis ...
7440,Payroll Accountant,Accounting,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Mid-Senior level,Bachelor's Degree,Full-time,payroll accountant focus primarily payroll fun...,desire fun -pron- genuine passion accounting p...


# TODO: combine two job posting datasets

In [ ]:
data_cleaned['Train'] = data_cleaned['description_cleaned'] + ' ' + data_cleaned['requirements_cleaned']
data_cleaned['Train']

0       company esri environmental systems research in...
1       job title itemization review manager location ...
2       -pron- responsibility manage english speak edi...
3       customer service associate base phoenix az rig...
4       position developer job location united states ...
                              ...                        
7437    avenue story look curious ambitious adaptable ...
7438    -pron- seek administrator work busy sale envir...
7439    -pron- activity search senior financial analys...
7440    payroll accountant focus primarily payroll fun...
7441    nemsia studios look experienced visual graphic...
Name: Train, Length: 7442, dtype: object

In [ ]:
data_test = data_cleaned['Train']

In [ ]:
data_test

0       company esri environmental systems research in...
1       job title itemization review manager location ...
2       -pron- responsibility manage english speak edi...
3       customer service associate base phoenix az rig...
4       position developer job location united states ...
                              ...                        
7437    avenue story look curious ambitious adaptable ...
7438    -pron- seek administrator work busy sale envir...
7439    -pron- activity search senior financial analys...
7440    payroll accountant focus primarily payroll fun...
7441    nemsia studios look experienced visual graphic...
Name: Train, Length: 7442, dtype: object

In [ ]:
vectorizer2 = TfidfVectorizer()
# Generate matrix of word vectors
tfidf_matrix = vectorizer2.fit_transform(data_test)

df_tfidf = pd.DataFrame(tfidf_matrix.toarray())
df_tfidf.columns = vectorizer2.get_feature_names_out()
df_tfidf

In [ ]:
df_tfidf.iloc[0]

aa                   0.0
aaa                  0.0
aabilities           0.0
aac                  0.0
aace                 0.0
                    ... 
zurb                 0.0
zurich               0.0
zusammenarbeitest    0.0
zynga                0.0
κnowledge            0.0
Name: 0, Length: 21446, dtype: float64

In [ ]:
# data_true_cc['required_education'].isnull().value_counts()

False    9360
True     7654
Name: required_education, dtype: int64

In [ ]:
# data_true_cc['required_experience'].isnull().value_counts()

False    10399
True      6615
Name: required_experience, dtype: int64

In [ ]:
# n = 0
# zr = range(len(data_true_cc['required_education'].isnull()))
# for z,i in zip(zr, data_true_cc['required_education'].isnull()):
#    if data_true_cc['required_experience'].isnull()[z]:
#      n+=1
# print(n)

6615


In [ ]:
# filter1 = data_true_cc['required_education'].isnull()
# filter2 = data_true_cc['required_experience'].isnull()
# filter3 = data_true_cc['employment_type'].isnull()
# filter4 = data_true_cc['requirements'].isnull()
# data_t_cc_l = data_true_cc.loc[~filter1].loc[~filter2].loc[~filter3].loc[~filter4]

In [ ]:
# data_t_cc_l.isnull().value_counts()


Unnamed: 0  index  title  department  description  requirements  required_experience  required_education  employment_type  description_c  requirements_c  Train
False       False  False  True        False        False         False                False               False            False          False           False    4538
                          False       False        False         False                False               False            False          False           False    2997
dtype: int64

In [ ]:
# data_t_cc_l['Train'].apply(str.encode("ascii"))

In [ ]:
# string_unicode = data_t_cc_l['Train']
# string_encode = string_unicode.encode("ascii", "ignore")
# string_decode = string_encode.decode()

In [ ]:
df_tfidf.columns.to_list()

In [ ]:
# pd.DataFrame([token.text for token in nlp(data_true['description'][0])])

In [ ]:
# len(tokens)

100

In [ ]:
# string[0]

'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staff'

In [ ]:
# tokens[1]

"Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90